In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import boto3
import rasterio
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
import os

In [27]:
df = pd.read_csv("Data/DroneVF_CalVal/DroneDataList.csv")
df.head(23)

,Date,Location Name,Nearest Town,Scan Start Time (UTC),Scan End Time (UTC)
0,3/21/2022,Fairland March,"Fairland, MD",2022-03-21T15:33Z,2022-03-21T15:41Z
1,6/4/2022,White 2,"White, SD",2022-06-04T15:13Z,2022-06-04T15:21Z
2,6/4/2022,Oak Lake,"Hendricks, SD",2022-06-04T16:13Z,2022-06-04T16:26Z
3,6/5/2022,Lisbon,"Englevale, ND",2022-06-05T16:30Z,2022-06-05T16:41Z
4,7/22/2022,Glen Ullin,"Glen Ullin, ND",2022-07-22T19:44Z,2022-07-22T19:51Z
5,7/25/2022,Velva,"Velva, ND",2022-07-25T17:12Z,2022-07-25T17:17Z
6,7/26/2022,Tioga,"Tioga, ND",2022-07-26T16:01Z,2022-07-26T16:05Z
7,8/1/2022,Island Park,"Ashton, ID",2022-08-01T17:22Z,2022-08-01T17:27Z
8,9/16/2022,Fairland Sept,"Fairland, MD",2022-09-16T19:22Z,2022-09-16T19:27Z
9,10/6/2022,Taylors Island,"Taylors Island, MD",2022-10-06T19:48Z,2022-10-06T19:55Z


In [4]:
imgList = []
for file in os.listdir("Data/DroneVF_CalVal/"):
    if file.endswith('.tif'):
        imgList.append(file)

['20220321_FairlandMarch_VF.tif', '20220604_OakLake_VF.tif', '20220604_White2_VF.tif', '20220605_Lisbon_VF.tif', '20220722_GlenUllin_VF.tif', '20220725_Velva_VF.tif', '20220726_Tioga_VF.tif', '20220801_IslandPark_VF.tif', '20220916_FairlandSept_VF.tif', '20221006_PoplarHill_VF.tif', '20221006_TaylorsIsland_VF.tif', '20221015_AguaFria_VF.tif', '20221018_Estrella_VF.tif', '20221018_PicachoPeak_VF.tif', '20221020_BuffaloPark_VF.tif', '20221020_LakeMary_VF.tif', '20221020_MormonLake2_VF.tif', '20221021_JosephCity2_VF.tif', '20221021_MeteorCrater2_VF.tif', '20221021_Winslow_VF.tif', '20221022_Holbrook_VF.tif']


In [39]:
geometry = []
img_file = []
for row in df.iterrows():
    location = row[1]['Location Name']
    date = row[1]['Date'].split('/')
    date = "".join([date[2],date[0].zfill(2),date[1].zfill(2)])
    #print("".join(location.split(' ')))
    #print(date)
    for img in imgList:
        if ("".join(location.split(' ')) in img) and (date in img):
            #print(img)
            img_file.append(img)
            with rasterio.open("Data/DroneVF_CalVal/"+img) as ds:
                bounds = ds.bounds
                crs_utm = ds.crs
            bounds_4326 = transform_bounds(crs_utm, CRS.from_epsg(4326), *bounds)
            classified_geometry = box(*bounds_4326)
            geometry.append(classified_geometry)

            #print(f'match found for {location} {date} {img}')
            

df['geometry'] = geometry
df['img_file'] = img_file
gdf = gpd.GeoDataFrame(df,geometry=geometry, crs= 4326)
gdf.head()

,Date,Location Name,Nearest Town,Scan Start Time (UTC),Scan End Time (UTC),geometry,img_file
0,3/21/2022,Fairland March,"Fairland, MD",2022-03-21T15:33Z,2022-03-21T15:41Z,"POLYGON ((-76.92651 39.07932, -76.92651 39.082...",20220321_FairlandMarch_VF.tif
1,6/4/2022,White 2,"White, SD",2022-06-04T15:13Z,2022-06-04T15:21Z,"POLYGON ((-96.66459 44.46127, -96.66459 44.465...",20220604_White2_VF.tif
2,6/4/2022,Oak Lake,"Hendricks, SD",2022-06-04T16:13Z,2022-06-04T16:26Z,"POLYGON ((-96.53132 44.50790, -96.53132 44.513...",20220604_OakLake_VF.tif
3,6/5/2022,Lisbon,"Englevale, ND",2022-06-05T16:30Z,2022-06-05T16:41Z,"POLYGON ((-97.94155 46.43370, -97.94155 46.437...",20220605_Lisbon_VF.tif
4,7/22/2022,Glen Ullin,"Glen Ullin, ND",2022-07-22T19:44Z,2022-07-22T19:51Z,"POLYGON ((-101.85755 46.70109, -101.85755 46.7...",20220722_GlenUllin_VF.tif
